# Diversificación de Portafolio

In [2874]:
# Santiago del Cid, 20210026
# Juan Pablo Galicia, 20210313
# José Andrés Segura, 20210137

In [2875]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from subprocess import check_output
import warnings
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
import datetime as dt
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from dash import dash_table
import yfinance as yf
import plotly.graph_objects as go

# 3. Analice los retornos por acción de 3 años y calcule las siguientes métricas:
    a.	Retorno medio
    b.	Retorno anualizado
    c.	Volatilidad
    d.	Sharpe
    e.	Curtosis
    f.	Sesgo (skewness)


# Selección de acciones

In [2878]:
# Amex, FedEx, GE, HomeDepot, Ferrari, Procter and Gamble
acciones = ['AXP','FDX','GE','HD','RACE','PG']

#3 años
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3*365) #3 años

#Data Frame para almacenar los datos
data = yf.download(acciones, start=start_date, end=end_date)["Adj Close"]
data

[*********************100%***********************]  6 of 6 completed


Ticker,AXP,FDX,GE,HD,PG,RACE
Date,,,,,,
2021-11-15 00:00:00+00:00,176.502487,237.631363,65.590103,344.140533,136.861221,253.659317
2021-11-16 00:00:00+00:00,174.372467,238.460205,63.548672,363.847778,136.666275,252.718567
2021-11-17 00:00:00+00:00,173.109894,232.969193,62.712410,366.184875,136.582733,260.940277
2021-11-18 00:00:00+00:00,169.832947,230.492050,61.900761,376.386322,136.601257,260.469940
2021-11-19 00:00:00+00:00,167.259552,228.580109,61.464188,379.020172,136.322754,265.869446
...,...,...,...,...,...,...
2024-11-07 00:00:00+00:00,286.820007,283.209991,178.850006,399.440002,163.410004,452.660004
2024-11-08 00:00:00+00:00,287.600006,286.279999,184.809998,405.899994,167.710007,454.459991
2024-11-11 00:00:00+00:00,292.970001,289.790009,184.559998,408.290009,166.029999,452.459991


# 2.	Grafique en un dashboard los precios y los retornos de las distintas acciones con un dropdown que permita seleccionar las acciones a mostrar, otro que permita elegir entre mostrar precios de cierre y los retornos y un slicer para filtrar las fechas.

# Cálcular los retornos utilizando tasa de variación

In [2881]:
returns = data.pct_change()
returns

Ticker,AXP,FDX,GE,HD,PG,RACE
Date,,,,,,
2021-11-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-16 00:00:00+00:00,-0.012068,0.003488,-0.031124,0.057265,-0.001424,-0.003709
2021-11-17 00:00:00+00:00,-0.007241,-0.023027,-0.013159,0.006423,-0.000611,0.032533
2021-11-18 00:00:00+00:00,-0.018930,-0.010633,-0.012942,0.027859,0.000136,-0.001802
2021-11-19 00:00:00+00:00,-0.015153,-0.008295,-0.007053,0.006998,-0.002039,0.020730
...,...,...,...,...,...,...
2024-11-07 00:00:00+00:00,-0.028256,0.011248,-0.010566,0.028504,0.014654,0.021806
2024-11-08 00:00:00+00:00,0.002719,0.010840,0.033324,0.016173,0.026314,0.003976
2024-11-11 00:00:00+00:00,0.018672,0.012261,-0.001353,0.005888,-0.010017,-0.004401


# Cálcular la media de los retornos diarios

In [2883]:
retorno_diario = returns.mean()
retorno_diario_df = pd.DataFrame(retorno_diario, columns = ['Retorno diario'])
retorno_diario_df

,Retorno diario
Ticker,
AXP,0.000823
FDX,0.000510
GE,0.001550
HD,0.000359
PG,0.000323
RACE,0.000892


In [2884]:
# Crear la aplicación de Dash
app = Dash(__name__)

# Layout del dashboard
app.layout = html.Div([
    html.H1("Análisis de Acciones"),

    # Dropdown para seleccionar acciones
    html.Label("Selecciona las acciones"),
    dcc.Dropdown(
        id="acciones-dropdown",
        options=[{"label": accion, "value": accion} for accion in acciones],
        multi=True,
        value=acciones  # Por defecto, mostrar todas
    ),

    # Dropdown para seleccionar entre precios de cierre y retornos
    html.Label("Mostrar datos de"),
    dcc.Dropdown(
        id="tipo-dato-dropdown",
        options=[
            {"label": "Precios de Cierre", "value": "Precios"},
            {"label": "Retornos", "value": "Retornos"}
        ],
        value="Precios"  # Por defecto, mostrar precios
    ),

    # Selector de rango de fechas
    html.Label("Selecciona el rango de fechas"),
    dcc.DatePickerRange(
        id="fecha-slicer",
        min_date_allowed=start_date,
        max_date_allowed=end_date,
        start_date=start_date,
        end_date=end_date
    ),

    # Gráfico de precios o retornos
    dcc.Graph(id="grafico-precios-retornos")
])

@app.callback(
    Output("grafico-precios-retornos", "figure"),
    [
        Input("acciones-dropdown", "value"),
        Input("tipo-dato-dropdown", "value"),
        Input("fecha-slicer", "start_date"),
        Input("fecha-slicer", "end_date")
    ]
)
def actualizar_grafico(acciones_seleccionadas, tipo_dato, fecha_inicio, fecha_fin):
    # Filtrar el rango de fechas seleccionado
    data_filtrada = data.loc[fecha_inicio:fecha_fin, acciones_seleccionadas]
    returns_filtrados = returns.loc[fecha_inicio:fecha_fin, acciones_seleccionadas]
    
    # Determinar si mostrar precios o retornos
    if tipo_dato == "Precios":
        fig = px.line(data_filtrada, x=data_filtrada.index, y=data_filtrada.columns,
                      labels={'value': 'Precio de Cierre', 'index': 'Fecha'},
                      title="Precios de Cierre de Acciones Seleccionadas")
    else:
        fig = px.line(returns_filtrados, x=returns_filtrados.index, y=returns_filtrados.columns,
                      labels={'value': 'Retorno Diario', 'index': 'Fecha'},
                      title="Retornos de Acciones Seleccionadas")
    
    # Ajustes adicionales para el gráfico
    fig.update_layout(xaxis_title="Fecha", yaxis_title="Valor")
    
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=2239)

# Cálcular los retornos anualizados

In [2886]:
# Crear un diccionario para almacenar los retornos anualizados de cada acción
annualized_returns = {}

# Calcular el retorno anualizado para cada acción
for ticker in acciones:
    # Calcular el retorno total en el periodo de observación
    total_return = (data[ticker].iloc[-1] - data[ticker].iloc[0]) / data[ticker].iloc[0]
    
    # Convertir a retorno anualizado
    annualized_return = ((1 + total_return) ** (12/36)) - 1
    
    # Guardar en el diccionario
    annualized_returns[ticker] = annualized_return

# Convertir el diccionario a un DataFrame para una visualización más clara
annualized_returns_df = pd.DataFrame.from_dict(annualized_returns, orient='index', columns=['Retorno Anualizado'])
annualized_returns_df


,Retorno Anualizado
AXP,0.176791
FDX,0.070956
GE,0.409224
HD,0.060270
RACE,0.201428
PG,0.067696


# Cálcular volatilidad

In [2888]:
vol = returns[acciones].std()*np.sqrt(250)
vol_df = pd.DataFrame(vol, columns=['Volatilidad'])
vol_df

,Volatilidad
Ticker,
AXP,0.295622
FDX,0.341301
GE,0.302023
HD,0.250226
RACE,0.284274
PG,0.176012


# Sharpe Ratio

In [2890]:
sharpe_ratio = ((annualized_returns_df['Retorno Anualizado']-0.01)/vol_df['Volatilidad'])
sharpe_ratio_df = pd.DataFrame(sharpe_ratio, columns = ['Sharpe Ratio'])
sharpe_ratio_df

,Sharpe Ratio
AXP,0.564205
FDX,0.178600
GE,1.321830
HD,0.200898
RACE,0.673393
PG,0.327794


# Curtosis

In [2892]:
# Calcular la curtosis de cada acción
kurtosis = returns.kurtosis()

# Convertir los resultados a un DataFrame para mejor visualización
kurtosis_df = pd.DataFrame(kurtosis, columns=['Curtosis'])
kurtosis_df

,Curtosis
Ticker,
AXP,3.783098
FDX,23.207274
GE,3.132206
HD,3.337004
PG,3.683045
RACE,4.767273


# Skewness

In [2894]:
# Calcular el sesgo de cada acción
skewness = returns.skew()

# Convertir los resultados a un DataFrame para mejor visualización
skewness_df = pd.DataFrame(skewness, columns=['Sesgo'])
skewness_df

,Sesgo
Ticker,
AXP,0.093540
FDX,-0.928315
GE,-0.277425
HD,-0.263240
PG,-0.451924
RACE,0.507729


# Visualización de métricas

In [2896]:
metricas = pd.concat([retorno_diario_df,annualized_returns_df,vol_df,sharpe_ratio_df,kurtosis_df,skewness_df], axis = 1, join = 'outer')
metricas

,Retorno diario,Retorno Anualizado,Volatilidad,Sharpe Ratio,Curtosis,Sesgo
AXP,0.000823,0.176791,0.295622,0.564205,3.783098,0.093540
FDX,0.000510,0.070956,0.341301,0.178600,23.207274,-0.928315
GE,0.001550,0.409224,0.302023,1.321830,3.132206,-0.277425
HD,0.000359,0.060270,0.250226,0.200898,3.337004,-0.263240
PG,0.000323,0.067696,0.176012,0.327794,3.683045,-0.451924
RACE,0.000892,0.201428,0.284274,0.673393,4.767273,0.507729


# 4. Arme un portafolio con las acciones recomendadas con los mismos pesos para todas y obtenga:
    a. retorno del portafolio
    b. retorno anualizado
    c. retorno histórico (acumulado)


# Acciones escogidas: AXP, GE, RACE

In [2899]:
# Amex, GE, Ferrari
portafolio = ['AXP','GE','RACE']

#3 años
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3*365) #3 años

#Data Frame para almacenar los datos
datos = yf.download(portafolio, start=start_date, end=end_date)["Adj Close"]
datos

[*********************100%***********************]  3 of 3 completed


Ticker,AXP,GE,RACE
Date,,,
2021-11-15 00:00:00+00:00,176.502487,65.590088,253.659317
2021-11-16 00:00:00+00:00,174.372452,63.548660,252.718567
2021-11-17 00:00:00+00:00,173.109894,62.712414,260.940308
2021-11-18 00:00:00+00:00,169.832916,61.900761,260.469940
2021-11-19 00:00:00+00:00,167.259552,61.464188,265.869446
...,...,...,...
2024-11-07 00:00:00+00:00,286.820007,178.850006,452.660004
2024-11-08 00:00:00+00:00,287.600006,184.809998,454.459991
2024-11-11 00:00:00+00:00,292.970001,184.559998,452.459991


# Retorno de las acciones

In [2901]:
returns2 = datos.pct_change()
returns2

Ticker,AXP,GE,RACE
Date,,,
2021-11-15 00:00:00+00:00,NaN,NaN,NaN
2021-11-16 00:00:00+00:00,-0.012068,-0.031124,-0.003709
2021-11-17 00:00:00+00:00,-0.007241,-0.013159,0.032533
2021-11-18 00:00:00+00:00,-0.018930,-0.012942,-0.001803
2021-11-19 00:00:00+00:00,-0.015152,-0.007053,0.020730
...,...,...,...
2024-11-07 00:00:00+00:00,-0.028256,-0.010566,0.021806
2024-11-08 00:00:00+00:00,0.002719,0.033324,0.003976
2024-11-11 00:00:00+00:00,0.018672,-0.001353,-0.004401


In [2902]:
meanDailyReturns = returns2.mean()
meanDailyReturns

Ticker
AXP     0.000823
GE      0.001550
RACE    0.000892
dtype: float64

# Retorno del portafolio diario

In [2904]:
#Retorno del portafolio diario
pesos = np.array([0.33,0.33,0.33])

#obtener retorno del portafolio con suma producto de los retornos c/u y los pesos
portReturns = np.sum(meanDailyReturns*pesos)
portReturns

0.0010775811124328935

# Retorno anualizado del portafolio

In [2906]:
# Valor inicial y final del portafolio
valor_inicial = np.sum(datos.iloc[0] * pesos)
valor_final = np.sum(datos.iloc[-1] * pesos)

# Retorno total del portafolio
retorno_total = (valor_final - valor_inicial) / valor_inicial

# Retorno anualizado del portafolio
retorno_anualizado_portafolio = ((1 + retorno_total) ** (12/36)) - 1
retorno_anualizado_portafolio

0.22489530714927475

# Retornos acumulados

In [2908]:
#Retornos acumulados
returns2['Portafolio'] = returns2.dot(pesos)
returns2

Ticker,AXP,GE,RACE,Portafolio
Date,,,,
2021-11-15 00:00:00+00:00,NaN,NaN,NaN,NaN
2021-11-16 00:00:00+00:00,-0.012068,-0.031124,-0.003709,-0.015477
2021-11-17 00:00:00+00:00,-0.007241,-0.013159,0.032533,0.004004
2021-11-18 00:00:00+00:00,-0.018930,-0.012942,-0.001803,-0.011113
2021-11-19 00:00:00+00:00,-0.015152,-0.007053,0.020730,-0.000487
...,...,...,...,...
2024-11-07 00:00:00+00:00,-0.028256,-0.010566,0.021806,-0.005615
2024-11-08 00:00:00+00:00,0.002719,0.033324,0.003976,0.013207
2024-11-11 00:00:00+00:00,0.018672,-0.001353,-0.004401,0.004263


In [2909]:
#Retornos acumulados
daily_cum_ret = (1+returns2).cumprod()
daily_cum_ret

Ticker,AXP,GE,RACE,Portafolio
Date,,,,
2021-11-15 00:00:00+00:00,NaN,NaN,NaN,NaN
2021-11-16 00:00:00+00:00,0.987932,0.968876,0.996291,0.984523
2021-11-17 00:00:00+00:00,0.980779,0.956126,1.028704,0.988465
2021-11-18 00:00:00+00:00,0.962213,0.943752,1.026849,0.977480
2021-11-19 00:00:00+00:00,0.947633,0.937096,1.048136,0.977004
...,...,...,...,...
2024-11-07 00:00:00+00:00,1.625020,2.726784,1.784520,2.075734
2024-11-08 00:00:00+00:00,1.629439,2.817651,1.791616,2.103147
2024-11-11 00:00:00+00:00,1.659863,2.813840,1.783731,2.112113


In [2910]:
# Retorno histórico acumulado al final del período
retorno_historico_acumulado = daily_cum_ret['Portafolio'].iloc[-1] - 1
retorno_historico_acumulado

1.0763318392010448

In [2911]:
portafolio_4 = np.array([portReturns, retorno_anualizado_portafolio, retorno_historico_acumulado]).reshape(3,1)
portafolio_4 = pd.DataFrame(portafolio_4, columns = ['Rendimiento'])
portafolio_4 = portafolio_4.rename(index={0:'Rendimiento diario', 1:'Rendimiento anualizado', 2:'Rendimiento Acumulado'})
portafolio_4

,Rendimiento
Rendimiento diario,0.001078
Rendimiento anualizado,0.224895
Rendimiento Acumulado,1.076332


# 5. Optimice el portafolio y extraiga:    
a.	Portafolio con el máximo sharpe    
b.	Portafolio con la varianza mínima


In [2913]:
import pyfolio as pf
import warnings
warnings.filterwarnings("ignore")
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Portafolio con el máximo sharpe

In [2915]:
mu = expected_returns.mean_historical_return(datos)
mu

Ticker
AXP     0.177555
GE      0.411151
RACE    0.202307
dtype: float64

In [2916]:
sigma = risk_models.sample_cov(datos)
sigma

Ticker,AXP,GE,RACE
Ticker,,,
AXP,0.088092,0.044557,0.037754
GE,0.044557,0.091948,0.034268
RACE,0.037754,0.034268,0.081458


In [2917]:
ef = EfficientFrontier(mu,sigma)
maxsharpe = ef.max_sharpe() #se saca la forntera ef del potafolio
maxsharpe

OrderedDict([('AXP', 0.0),
             ('GE', 0.8840780729673666),
             ('RACE', 0.1159219270326334)])

In [2918]:
weights_maxsharpe = ef.clean_weights() #extraer solo los pesos de ese portafolio
weights_maxsharpe

OrderedDict([('AXP', 0.0), ('GE', 0.88408), ('RACE', 0.11592)])

In [2919]:
pesos_sharpe = np.array([0.00,0.88408,0.11592])
# Valor inicial y final del portafolio
valor_inicial = np.sum(datos.iloc[0] * pesos_sharpe)
valor_final = np.sum(datos.iloc[-1] * pesos_sharpe)

# Retorno total del portafolio
retorno_total = (valor_final - valor_inicial) / valor_inicial

# Retorno anualizado del portafolio
retorno_anualizado_portafolio_sharpe = ((1 + retorno_total) ** (12/36)) - 1
retorno_anualizado_portafolio_sharpe

0.34634633874803056

# Portafolio con minima volatilidad

In [2921]:
ef = EfficientFrontier(mu,sigma)
minvol = ef.min_volatility()
weights_minvol = ef.clean_weights()
weights_minvol

OrderedDict([('AXP', 0.29298), ('GE', 0.29914), ('RACE', 0.40787)])

In [2922]:
pesos_vol = np.array([0.29298,0.29915,0.40787])
# Valor inicial y final del portafolio
valor_inicial = np.sum(datos.iloc[0] * pesos_vol)
valor_final = np.sum(datos.iloc[-1] * pesos_vol)

# Retorno total del portafolio
retorno_total = (valor_final - valor_inicial) / valor_inicial

# Retorno anualizado del portafolio
retorno_anualizado_portafolio_vol = ((1 + retorno_total) ** (12/36)) - 1
retorno_anualizado_portafolio_vol

0.22154217435519463

In [2923]:
retornos = np.array([retorno_anualizado_portafolio, retorno_anualizado_portafolio_sharpe, retorno_anualizado_portafolio_vol])
retornos.reshape(1,3)

array([[0.22489531, 0.34634634, 0.22154217]])

In [2924]:
retornos = pd.DataFrame([retornos], columns = ['Potafolio 1','Portafolio sharpe','Portafolio volatility'])
retornos = retornos.rename(index={0:'Rendimiento anualizado'})
retornos

,Potafolio 1,Portafolio sharpe,Portafolio volatility
Rendimiento anualizado,0.224895,0.346346,0.221542


# Retornos anualizados de los 3 portafolios

In [2926]:
retornos = retornos.rename(index={0:'Retorno Anualizado'})
retornos

,Potafolio 1,Portafolio sharpe,Portafolio volatility
Rendimiento anualizado,0.224895,0.346346,0.221542


In [2927]:
# Definir el conjunto de acciones y fechas de inicio y fin
stocks = ['AXP', 'GE', 'RACE']
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3*365)  # Últimos 3 años

# Descargar datos de precios ajustados y calcular retornos diarios
data = yf.download(stocks, start=start_date, end=end_date)["Adj Close"]
returns2 = data.pct_change()

# Definir los pesos para los dos portafolios
w2 = np.array([0.00, 0.88408, 0.11592])  # Portafolio Max Sharpe
w3 = np.array([0.29298, 0.29915, 0.40787])  # Portafolio Min Volatilidad

# Seleccionar solo las columnas de las acciones para evitar el error de dimensiones
returns_stocks = returns2[stocks]

# Calcular retornos del portafolio con pesos w2 y w3
returns2["Portfolio_max_sharpe"] = returns_stocks.dot(w2)
returns2["Portfolio_min_vol"] = returns_stocks.dot(w3)

# Crear la aplicación de Dash
app1 = Dash(__name__)

# Layout del dashboard
app1.layout = html.Div([
    html.H1("Análisis de Portafolios"),

    # Dropdown para seleccionar el tipo de portafolio
    html.Label("Selecciona el portafolio para ver la distribución de retornos"),
    dcc.Dropdown(
        id="portafolio-dropdown",
        options=[
            {"label": "Portafolio Max Sharpe", "value": "Portfolio_max_sharpe"},
            {"label": "Portafolio Min Volatilidad", "value": "Portfolio_min_vol"}
        ],
        value="Portfolio_max_sharpe"  # Valor inicial
    ),

    # Gráfico de distribución de los retornos del portafolio seleccionado
    dcc.Graph(id="grafico-distribucion-retornos")
])

@app1.callback(
    Output("grafico-distribucion-retornos", "figure"),
    [Input("portafolio-dropdown", "value")]
)
def actualizar_grafico_distribucion(portafolio_seleccionado):
    # Crear el histograma de la distribución de retornos del portafolio seleccionado
    fig = px.histogram(
        returns2,
        x=portafolio_seleccionado,
        nbins=50,
        title=f"Distribución de Retornos - {portafolio_seleccionado.replace('_', ' ').title()}",
        labels={portafolio_seleccionado: "Retorno Diario"}
    )
    fig.update_layout(xaxis_title="Retorno Diario", yaxis_title="Frecuencia")
    
    return fig

if __name__ == "__main__":
    app1.run_server(debug=True, port=8051)

[*********************100%***********************]  3 of 3 completed


In [2962]:
# Definir el conjunto de acciones y fechas de inicio y fin
stocks = ['AXP', 'GE', 'RACE']
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3*365)  # Últimos 3 años

# Descargar datos de precios ajustados y calcular retornos diarios
data = yf.download(stocks, start=start_date, end=end_date)["Adj Close"]
returns2 = data.pct_change()

# Definir los pesos para los dos portafolios
w2 = np.array([0.00, 0.88408, 0.11592])  # Portafolio Max Sharpe
w3 = np.array([0.29298, 0.29915, 0.40787])  # Portafolio Min Volatilidad

# Seleccionar solo las columnas de las acciones para evitar el error de dimensiones
returns_stocks = returns2[stocks]

# Calcular retornos del portafolio con pesos w2 y w3
returns2["Portfolio_max_sharpe"] = returns_stocks.dot(w2)
returns2["Portfolio_min_vol"] = returns_stocks.dot(w3)

# Calcular el rendimiento acumulado para cada portafolio
returns2["Portfolio_max_sharpe_cum"] = (1 + returns2["Portfolio_max_sharpe"]).cumprod()
returns2["Portfolio_min_vol_cum"] = (1 + returns2["Portfolio_min_vol"]).cumprod()

# Crear la aplicación de Dash
app1 = Dash(__name__)

# Layout del dashboard
app1.layout = html.Div([
    html.H1("Análisis de Portafolios"),

    # Intervalo para actualizar el gráfico cada vez que la página se carga
    dcc.Interval(id="intervalo", interval=1*1000, n_intervals=0),  # Intervalo de 1 segundo para cargar inicialmente

    # Gráfico de rendimiento acumulado de los portafolios
    html.H2("Rendimiento Acumulado de los Portafolios"),
    dcc.Graph(id="grafico-rendimiento-acumulado")
])

@app1.callback(
    Output("grafico-rendimiento-acumulado", "figure"),
    [Input("intervalo", "n_intervals")]
)
def actualizar_grafico_rendimiento_acumulado2(n_intervals):
    # Gráfico de rendimiento acumulado de los portafolios
    fig_cum = go.Figure()
    fig_cum.add_trace(go.Scatter(
        x=returns2.index,
        y=returns2["Portfolio_max_sharpe_cum"],
        mode='lines',
        name="Portafolio Max Sharpe"
    ))
    fig_cum.add_trace(go.Scatter(
        x=returns2.index,
        y=returns2["Portfolio_min_vol_cum"],
        mode='lines',
        name="Portafolio Min Volatilidad"
    ))

    fig_cum.update_layout(
        title="Rendimiento Acumulado de los Portafolios",
        xaxis_title="Fecha",
        yaxis_title="Rendimiento Acumulado",
    )
    
    return fig_cum

if __name__ == "__main__":
    app1.run_server(debug=True, port=8051)

[*********************100%***********************]  3 of 3 completed
